# FuzzyART

Aqui, embaralharemos os dados e avaliaremos o modelo usando a métrica ARI.

## Importando bibliotecas

In [1]:
import torch
from torchvision import datasets, transforms
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from artlib import FuzzyART
from sklearn.metrics.cluster import adjusted_rand_score
import pandas as pd

## Baixando base de dados

In [2]:
transform = transforms.ToTensor()
dataset = datasets.USPS(root='./data', train = True, download=True, transform=transform)

X = np.stack([img.numpy() for img, _ in dataset])
y = np.array([label for _, label in dataset])

X, y = shuffle(X, y, random_state=42)

print(X.shape, y.shape)

(7291, 1, 16, 16) (7291,)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X_test.shape, X_train.shape)
print(y_test.shape, y_train.shape)

(2917, 1, 16, 16) (4374, 1, 16, 16)
(2917,) (4374,)


## Treinando o modelo

In [4]:
alphas = [0.0] 
rhos   = [0.3]
betas  = [1.0]

results = []

for alpha in alphas:
    for rho in rhos:
        for beta in betas:

            model = FuzzyART(rho=rho, alpha=alpha, beta=beta)
            
            lower_bound, upper_bound = model.find_data_bounds(X)
            model.set_data_bounds(lower_bound, upper_bound)

            # Preparar os dados (normalizados internamente pelo modelo)
            train_X_prep = model.prepare_data(X_train)
            test_X_prep  = model.prepare_data(X_test)

            # Treinar
            model.fit(train_X_prep, y_train)

            # Prever
            predictions = model.predict(test_X_prep)

            # Calcular accuracy
            ari = adjusted_rand_score(y_test, predictions)

            results.append({
                'alpha': alpha,
                'rho': rho,
                'beta': beta,
                'ARI': ari
            })

            #print(f"alpha={alpha:.1e}, rho={rho}, beta={beta} → acc={acc:.3f}")

df_results = pd.DataFrame(results)
best_results = df_results.sort_values(by='ARI', ascending=False).head(10)


In [5]:
print("10 melhores combinações:")
print(best_results)

10 melhores combinações:
   alpha  rho  beta  ARI
0    0.0  0.3   1.0  0.0
